Task 3

Using Python convert data from task2 into  Parquet and save as a copy 

(For example for Java check https://hadoop.apache.org/docs/r2.7.5/api/org/apache/hadoop/fs/FileSystem.html). 

Copy scripts to the archive. Make screenshot of parquet contents.

While uploading large files directly in HDFS via standard tools you'll get some problems due to file size. The workaround is to use batch loading (requires additional scripts in Python/Java/Scala)

https://towardsdatascience.com/why-and-how-to-use-pandas-with-large-data-9594dda2ea4c

EXPECTED OUTPUTS:
·         ZIP-ed folder with your screenshots


# Convert using Spark

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd

Make list of used files

In [9]:
root = 'hdfs://172.17.0.2/user/hive/warehouse'

files = ('destinations',
          'sample_submission',
          'test',
          'train')

csvs = [f'{root}/{file}/{file}.csv' for file in files]

csvs              

['hdfs://172.17.0.2/user/hive/warehouse/destinations/destinations.csv',
 'hdfs://172.17.0.2/user/hive/warehouse/sample_submission/sample_submission.csv',
 'hdfs://172.17.0.2/user/hive/warehouse/test/test.csv',
 'hdfs://172.17.0.2/user/hive/warehouse/train/train.csv']

Start Spark session

In [10]:
spark = SparkSession.builder.appName("read-csv-write-parquet").getOrCreate()
spark

Define function for reading data

In [11]:
def read_csv_from_hdfs(path):
    """Reads csv from HDFS as Spark DF"""
    df_load = spark.read.csv(path, header = 'true', inferSchema = True)
    return(df_load)

In [23]:
#optionally check read function
read_csv_from_hdfs(csvs[1]).show(5)

+---+-------------+
| id|hotel_cluster|
+---+-------------+
|  0|         99 1|
|  1|         99 1|
|  2|         99 1|
|  3|         99 1|
|  4|         99 1|
+---+-------------+
only showing top 5 rows



In [25]:
def write_spark_df_to_parquet(spark_df, location):
    return spark_df.write.parquet(location + '.parquet')

In [26]:
for file in files:
    in_location = f'{root}/{file}/{file}.csv'
    out_location = f'staging/{file}'
    print('reading', in_location)
    write_spark_df_to_parquet(read_csv_from_hdfs(in_location), out_location)
    print(f'writed to /{out_location}.parquet')

reading hdfs://172.17.0.2/user/hive/warehouse/destinations/destinations.csv


writed to /staging/destinations.parquet
reading hdfs://172.17.0.2/user/hive/warehouse/sample_submission/sample_submission.csv


writed to /staging/sample_submission.parquet
reading hdfs://172.17.0.2/user/hive/warehouse/test/test.csv


writed to /staging/test.parquet
reading hdfs://172.17.0.2/user/hive/warehouse/train/train.csv


writed to /staging/train.parquet


Show folder content

In [27]:
!ls  staging/*

staging/destinations.parquet:
part-00000-bc234cab-875f-403c-8ef6-71ad645f23e5-c000.snappy.parquet
part-00001-bc234cab-875f-403c-8ef6-71ad645f23e5-c000.snappy.parquet
part-00002-bc234cab-875f-403c-8ef6-71ad645f23e5-c000.snappy.parquet
part-00003-bc234cab-875f-403c-8ef6-71ad645f23e5-c000.snappy.parquet
_SUCCESS

staging/sample_submission.parquet:
part-00000-41c102ab-7cc3-4ae0-a8e9-40210cb61966-c000.snappy.parquet
part-00001-41c102ab-7cc3-4ae0-a8e9-40210cb61966-c000.snappy.parquet
part-00002-41c102ab-7cc3-4ae0-a8e9-40210cb61966-c000.snappy.parquet
part-00003-41c102ab-7cc3-4ae0-a8e9-40210cb61966-c000.snappy.parquet
_SUCCESS

staging/test.parquet:
part-00000-c43028ea-a928-4519-b8f5-b4400ba51942-c000.snappy.parquet
part-00001-c43028ea-a928-4519-b8f5-b4400ba51942-c000.snappy.parquet
part-00002-c43028ea-a928-4519-b8f5-b4400ba51942-c000.snappy.parquet
part-00003-c43028ea-a928-4519-b8f5-b4400ba51942-c000.snappy.parquet
_SUCCESS

staging/train.parquet:
part-00000-2a0f62c0-c22d-4c97-a4b4-87941a40c

Show binary content of one of parquet files

In [28]:
!echo ================== head of binary parquet =========================
!head staging/train.parquet/part-00002-2a0f62c0-c22d-4c97-a4b4-87941a40cba0-c000.snappy.parquet
!echo ================== tail of binary parquet =========================
!tail staging/train.parquet/part-00002-2a0f62c0-c22d-4c97-a4b4-87941a40cba0-c000.snappy.parquet

================== head of binary parquet =========================
.��=222� 7.�33Yms6A	. 7.$a� 5�xa%�3-0����P07a� 5*�12G09A*	 6y�4-06m 4�	-B�"q�Im�	52�	. 6��04.o	\12�g.A?��6-	.32.� 7e� 2>\ 	 5.*	06 0%Y1B10 6:�.  92J 7��� 8E� 8e%N  59 4A?24�� 52�077:01.a6:0]�.���05-09�G 2��J312�.33:5q< 4. 5�x 3aj01��57yS6 .m 4.6:07�9.6:%�
.  22� 612:24� 3!B!2:2M31&�
6  3Q(.  5&3-11!�10��*�
01s 5
. 6!�*02�� 5e�Ѧ 46:18.�
�����q� 42 :�
 32�.10.� 1�x2� 	 2*5
 3�� 4 4.. 5.�
.  99p 3��41Y��003�� 5�.  8Y19a�43�J.  5��.  7�J�G12�3 1��.  4.�08. 52�x�	E132YE 5yj.. 2�a%]o	s30.V 6�182�	 3A�L>�	. 26E �08:21&��9 9��.  3&�. *�
 62�  82f.� 8�
================== tail of binary parquet =========================
   &�Ґsrch_destination_id�ї���̫�&�Ґ<�      (�      ,  
   &���srch_destination_type_id

Show content in pandas dataframe 

In [29]:
pd.read_parquet('staging/destinations.parquet')

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690
2,2,-2.183490,-2.224164,-2.224164,-2.189562,-2.105819,-2.075407,-2.224164,-2.118483,-2.140393,...,-2.224164,-2.224164,-2.196379,-2.224164,-2.192009,-2.224164,-2.224164,-2.224164,-2.224164,-2.057548
3,3,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.115485,-2.177409,-2.177409,-2.177409,...,-2.161081,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409
4,4,-2.189562,-2.187783,-2.194008,-2.171153,-2.152303,-2.056618,-2.194008,-2.194008,-2.145911,...,-2.187356,-2.194008,-2.191779,-2.194008,-2.194008,-2.185161,-2.194008,-2.194008,-2.194008,-2.188037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62101,64989,-2.145876,-2.158016,-2.216621,-2.158016,-2.006209,-2.102505,-2.216621,-2.157203,-2.216621,...,-2.216621,-2.158016,-2.158016,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.152024
62102,64990,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.176802,-2.203479,-2.203479,-2.203479,...,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.008885
62103,64991,-2.190567,-2.208308,-2.208308,-2.208308,-2.208308,-2.135493,-2.208308,-2.208308,-1.909808,...,-2.188935,-2.208308,-2.175594,-2.208308,-2.208308,-2.199347,-2.208308,-2.208308,-2.208308,-2.208308
62104,64992,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,...,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686
